This notebook contains code I used to help with working out some of the analysis.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.gridspec as gridspec
plt.style.use('seaborn-paper')
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

In [ ]:
beta = 0.3
gamma = 0.1
sigma = beta/gamma
N1 = 10; N2=5
Y, X = np.mgrid[0:1:100j, 0:1:100j]
U = -beta*X*Y
V = beta*X*Y - gamma*Y
x_points = list(np.linspace(0,1,N1)) + list(np.linspace(1./sigma,1,N2))
y_points = list(1.-np.linspace(0,1,N1)) + [1.e-6]*N2
seed_points = np.array([x_points, y_points])

plt.figure(figsize=(6,6))
plt.streamplot(X, Y, U, V, start_points=seed_points.T,integration_direction='forward',maxlength=1000,broken_streamlines=False)
plt.plot([0,1],[1,0],'-k')
plt.plot([1./sigma,1./sigma],[0,1-1./sigma],'--k',alpha=0.5)
plt.axis('image');
plt.xlim(0,1);  plt.ylim(0,1);
plt.xlabel('x'); plt.ylabel('y');
plt.savefig('sigma15.pdf')

In [ ]:
from scipy.special import lambertw

In [ ]:
def x_inf(x,y,sigma):
    return -1./sigma * np.real(lambertw(-x*sigma*np.exp(-sigma*(x+y))))

In [ ]:
x_inf(0.6,1.e-3,3.)

In [ ]:
x_inf(0.99,1.e-3,3.)

In [ ]:
def dxinf_dy(x0,y0,sigma):
    return -1./(np.exp(sigma*(x0+y0-x_inf(x0,y0,sigma)))/(x0*sigma)-1)

In [ ]:
x0 = 0.1
y0 = np.linspace(0.0001,1-x0)
plt.plot(y0,-dxinf_dy(0.5,y0,3)*y0)

In [ ]:
x0 = np.linspace(0.1,0.6)
y0 = 0.1
plt.plot(x0,dxinf_dy(x0,y0,3))

In [ ]:
plt.contour(X,Y,x_inf(X,Y,3))
plt.axis('image')

In [ ]:
Y, X = np.mgrid[1e-2:1:100j, 1e-2:1:100j]

plt.contourf(X,Y,dxinf_dy(X,Y,3))
plt.colorbar()
plt.contour(X,Y,x_inf(X,Y,3))
plt.axis('image')

In [ ]:
from scipy.integrate import solve_ivp

def SIR_bangbang(beta=0.3, gamma=0.1, x0=0.99, y0=0.01, switch_times=[],T=100):
    """ Model the current outbreak using the SIR model."""

    du = np.zeros(3)
    u0 = np.zeros(3)
    switch_times = np.array(switch_times)
    
    def f(t,u):
        if np.argmax(t<switch_times) % 2 == 0:
            qval = 0
        else:
            qval = 1
        du[0] = -(1-qval)*beta*u[1]*u[0]
        du[1] = (1-qval)*beta*u[1]*u[0] - gamma*u[1]
        return du

    # Initial values
    u0[1] = y0 # Initial infected
    u0[0] = x0

    #dt = 0.01
    times = np.linspace(0,T,1000)
    solution = solve_ivp(f,[0,T],u0,t_eval=times,method='RK23',max_step=0.1)
    S = solution.y[0,:]
    I = solution.y[1,:]
    
    fig, ax = plt.subplots(1,2)
    ax[0].plot(S,'-',lw=3)
    ax[0].plot(I,'-',lw=3)
    ax[0].legend(['Susceptible','Infected',r'$x_\infty$'])

    
    N1 = 10; N2=5
    Y, X = np.mgrid[0:1:100j, 0:1:100j]
    U = -beta*X*Y
    V = beta*X*Y - gamma*Y
    x_points = list(np.linspace(0,1,N1)) + list(np.linspace(1./sigma,1,N2))
    y_points = list(1.-np.linspace(0,1,N1)) + [1.e-6]*N2
    seed_points = np.array([x_points, y_points])

    ax[1].streamplot(X, Y, U, V, start_points=seed_points.T,integration_direction='forward',maxlength=1000,broken_streamlines=False)
    ax[1].plot([0,1],[1,0],'-k')
    ax[1].plot([1./sigma,1./sigma],[0,1-1./sigma],'--k',alpha=0.5)
    ax[1].axis('image');
    ax[1].set_xlabel('x'); ax[1].set_ylabel('y');
    ax[1].plot(S,I,lw=3); ax[1].set_xlim(0,1); ax[1].set_ylim(0,1)
    ax[1].plot([gamma/beta, gamma/beta],[0,1],'--k',alpha=0.5)
    return S, I

In [ ]:
x0 = 0.7; y0 = 0.2
S1, I1 = SIR_bangbang(x0=x0,y0=y0,switch_times=[0,25],T=54)

In [ ]:
S2, I2 = SIR_bangbang(x0=x0,y0=y0,switch_times=[8,16.5],T=16.5)

In [ ]:
switch_times = [2,4,6,8,10,12,14,16,18,20,21.6]
S3, I3 = SIR_bangbang(x0=x0,y0=y0,switch_times=switch_times,T=switch_times[-1])

In [ ]:
N1 = 10; N2=5
Y, X = np.mgrid[0:1:100j, 0:1:100j]
U = -beta*X*Y
V = beta*X*Y - gamma*Y
x_points = list(np.linspace(0,1,N1)) + list(np.linspace(1./sigma,1,N2))
y_points = list(1.-np.linspace(0,1,N1)) + [1.e-6]*N2
seed_points = np.array([x_points, y_points])

plt.figure(figsize=(6,6))
plt.streamplot(X, Y, U, V, start_points=seed_points.T,integration_direction='forward',maxlength=1000,broken_streamlines=False)
plt.plot([0,1],[1,0],'-k')
plt.plot([1./sigma,1./sigma],[0,1-1./sigma],'--k',alpha=0.5)
plt.axis('image');
plt.xlim(0,1);  plt.ylim(0,1);
plt.xlabel('x'); plt.ylabel('y');
plt.plot(S1,I1,lw=3);
plt.plot(S2,I2,lw=3);
plt.plot(S3,I3,lw=3);
plt.savefig('threepaths.pdf')

In [ ]:
from scipy.integrate import solve_ivp

def SIR_general_control(beta=0.3, gamma=0.1, x0=0.99, y0=0.01, qfun=None, T=100):
    """ Model the current outbreak using the SIR model."""

    du = np.zeros(3)
    u0 = np.zeros(3)
    
    def f(t,u):
        qval = qfun(t,u)
        du[0] = -(1-qval)*beta*u[1]*u[0]
        du[1] = (1-qval)*beta*u[1]*u[0] - gamma*u[1]
        return du

    # Initial values
    u0[1] = y0 # Initial infected
    u0[0] = x0

    #dt = 0.01
    times = np.linspace(0,T,10000)
    solution = solve_ivp(f,[0,T],u0,t_eval=times,method='RK23',max_step=0.1)
    S = solution.y[0,:]
    I = solution.y[1,:]
    
    fig, ax = plt.subplots(1,2)
    ax[0].plot(S,'-',lw=3)
    ax[0].plot(I,'-',lw=3)
    ax[0].legend(['Susceptible','Infected',r'$x_\infty$'])

    
    N1 = 10; N2=5
    Y, X = np.mgrid[0:1:100j, 0:1:100j]
    U = -beta*X*Y
    V = beta*X*Y - gamma*Y
    x_points = list(np.linspace(0,1,N1)) + list(np.linspace(1./sigma,1,N2))
    y_points = list(1.-np.linspace(0,1,N1)) + [1.e-6]*N2
    seed_points = np.array([x_points, y_points])

    ax[1].streamplot(X, Y, U, V, start_points=seed_points.T,integration_direction='forward',maxlength=1000,broken_streamlines=False)
    ax[1].plot([0,1],[1,0],'-k')
    ax[1].plot([1./sigma,1./sigma],[0,1-1./sigma],'--k',alpha=0.5)
    ax[1].axis('image');
    ax[1].set_xlabel('x'); ax[1].set_ylabel('y');
    ax[1].plot(S,I,lw=3); ax[1].set_xlim(0,1); ax[1].set_ylim(0,1)
    ax[1].plot([gamma/beta, gamma/beta],[0,1],'--k',alpha=0.5)
    return S, I

In [ ]:
x0=0.99
y0 = 0.01
def xinf_sigma(sigma):
    return x_inf(x0, y0, sigma)
from scipy.optimize import fsolve

sigma0 = 3.
myfun = lambda sigma: xinf_sigma(sigma) - 1/sigma0
sigmastar = fsolve(myfun,sigma0)[0]
qfun = lambda t, u: 1-sigmastar/sigma0
1-sigmastar/sigma0

In [ ]:
S1, I1 = SIR_general_control(qfun=qfun,T=1000)

In [ ]:
def qfun(t,u):
    if u[0]>1./sigma0: return 0
    else: return 1
    
S2, I2 = SIR_general_control(qfun=qfun,T=1000)

In [ ]:
N1 = 10; N2=5
Y, X = np.mgrid[0:1:100j, 0:1:100j]
U = -beta*X*Y
V = beta*X*Y - gamma*Y
x_points = list(np.linspace(0,1,N1)) + list(np.linspace(1./sigma,1,N2))
y_points = list(1.-np.linspace(0,1,N1)) + [1.e-6]*N2
seed_points = np.array([x_points, y_points])

plt.figure(figsize=(6,6))
plt.streamplot(X, Y, U, V, start_points=seed_points.T,integration_direction='forward',maxlength=1000,broken_streamlines=False)
plt.plot([0,1],[1,0],'-k')
plt.plot([1./sigma,1./sigma],[0,1-1./sigma],'--k',alpha=0.5)
plt.axis('image');
plt.xlim(0,1);  plt.ylim(0,1);
plt.xlabel('x'); plt.ylabel('y');
plt.plot(S1,I1,lw=3,label='Constant control');
plt.plot(S2,I2,lw=3, label='Bang-bang control');
plt.legend()
plt.savefig('twocontrols.pdf')